In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'july'
new_month = 'august'

In [38]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
                         sheetname='assets',index_col='Ticker')
#change call to new filepath
#coinmarketcap_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2019.xlsx',\
#                         sheetname='coinmarketcap',index_col='Ticker')

In [4]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [5]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.loc['VEN'] = df.loc['VET']
df.index = map(lambda x: x.lower(), df.index)

In [6]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [7]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Available Supply'] = (d_messari['data']['supply']['circulating'])
    df_all['Supply % Issued'] = df_all['Available Supply'] / (df_all['Y2050 Supply'])
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Available Supply']
    df_all['Category'] = (d_messari['data']['misc_data']['categories'][0])
    df_all['Sectors'] = (d_messari['data']['misc_data']['sectors'][0])
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    except:
        pass
    try:
        df_all['Category2'] = (d_messari['data']['misc_data']['categories'][1])
    except:
        pass
    try:
        df_all['Sectors2'] = (d_messari['data']['misc_data']['sectors'][1])
    except:
        pass
    return df_all

In [8]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Available Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        df.loc[t,('Category')] = metrics['Category']
        df.loc[t,('Sectors')] = metrics['Sectors']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
        try:
            df.loc[t,('Category2')] = metrics['Category2']
        except:
            pass
        try:
            df.loc[t,('Sectors2')] = metrics['Sectors2']
        except:
            pass
    except:
        pass

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [9]:
print(metrics_log['btc'])

Y2050 Supply         2.09863e+07
Available Supply     1.78477e+07
Supply % Issued         0.850444
Price                    9682.57
Marketcap            2.03202e+11
Current Marketcap    1.72812e+11
Category                currency
Sectors                 currency
Est Volume           1.39932e+11
dtype: object


In [10]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [12]:
t = 'VEN'
metrics = get_messari_data(t)
metrics_log[t] = metrics
df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
df.loc[t,('Price')] = metrics['Price']
df.loc[t,('Marketcap')] = metrics['Marketcap']
df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
df.loc[t,('Category')] = metrics['Category']
df.loc[t,('Sectors')] = metrics['Sectors']
try:
    df.loc[t,('Est Volume')] = metrics['Est Volume']
except:
    pass
try:
    df.loc[t,('Category2')] = metrics['Category2']
except:
    pass
try:
    df.loc[t,('Sectors2')] = metrics['Sectors2']
except:
   pass

In [13]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [14]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [15]:
df.loc['IOTA']

Coin                           IOTA
Class                             1
Sector                          NaN
Quality Exchange?                 1
ERC20                             0
ICO ok?                           1
Eligible                          1
Y2050 Supply            2.77953e+09
Available Supply        2.77953e+09
Supply % Issued                   1
Price                       0.28699
Marketcap               7.97699e+08
Current Marketcap       7.97699e+08
Category             infrastructure
Sectors                         iot
Est Volume               2.3148e+08
Sectors2                        NaN
Category2                       NaN
Name: IOTA, dtype: object

In [16]:
#read in other data sources
onchainfx_db = df

In [17]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

146
146
145
145


In [18]:
#fix any ticker mapping issues
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [44]:
onchainfx_db

,Coin,Class,Sector,Quality Exchange?,ERC20,ICO ok?,Eligible,Y2050 Supply,Available Supply,Supply % Issued,Price,Marketcap,Current Marketcap,Category,Sectors,Est Volume,Sectors2,Category2
ADA,Cardano,2.0,NaN,1.0,0.0,1.0,1.0,4.201402e+10,2.592707e+10,0.617105,0.060445,2.539518e+09,1.567149e+09,infrastructure,smart_contract_platform,6.677349e+08,NaN,NaN
AE,Aeternity,3.0,NaN,1.0,1.0,1.0,1.0,5.036164e+08,2.765723e+08,0.549173,0.308271,1.552505e+08,8.525931e+07,infrastructure,smart_contract_platform,3.747520e+08,NaN,NaN
AION,Aion,2.0,NaN,1.0,1.0,1.0,1.0,6.410953e+08,3.273044e+08,0.510539,0.114327,7.329436e+07,3.741966e+07,infrastructure,smart_contract_platform,5.841537e+07,interoperability,NaN
ANT,Aragon,3.0,NaN,1.0,1.0,1.0,1.0,3.960952e+07,3.019856e+07,0.762407,0.841977,3.335032e+07,2.542650e+07,infrastructure,governance,2.279824e+05,NaN,NaN
ARDR,Ardor,2.0,NaN,1.0,0.0,1.0,1.0,9.989995e+08,9.989995e+08,1.000000,0.068529,6.846082e+07,6.846082e+07,infrastructure,enterprise_baas,7.380627e+06,NaN,NaN
ARK,Ark,2.0,NaN,1.0,0.0,1.0,1.0,3.819367e+08,1.134435e+08,0.297022,0.354206,1.352843e+08,4.018236e+07,infrastructure,interoperability,3.306358e+07,NaN,NaN
BAT,BAT,3.0,NaN,1.0,1.0,1.0,1.0,1.500000e+09,1.274571e+09,0.849714,0.248567,3.728500e+08,3.168157e+08,media_and_entertainment,ads,9.136809e+07,NaN,NaN
BCD,Bitcoin Diamond,1.0,NaN,1.0,0.0,1.0,1.0,2.100000e+08,1.864929e+08,0.888061,0.814639,1.710741e+08,1.519243e+08,currency,currency,3.563784e+07,NaN,NaN
BCH,Bitcoin Cash,1.0,NaN,1.0,0.0,1.0,1.0,2.098243e+07,1.792051e+07,0.854072,330.169310,6.927756e+09,5.916803e+09,currency,currency,8.514442e+09,NaN,NaN
BCN,Bytecoin,1.0,NaN,1.0,0.0,0.0,0.0,1.844675e+11,1.840668e+11,0.997828,0.000577,1.064671e+08,1.062359e+08,currency,currency,2.859959e+05,privacy,NaN


In [20]:
#onchainfx_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\ocfx_'+new_month+'_2019.xlsx',\
#index_col='Ticker')

In [21]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [22]:
#main function to create table of all assets
#def create_universe(onchainfx,coinmarketcap,assets):
#    eligibility = pd.DataFrame(columns = ['Marketcap'])
#    eligibility['Marketcap'] = onchainfx['Marketcap']
#    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
#    eligibility['Float Percent'] = onchainfx['Supply % Issued']
#    eligibility['Float'] = onchainfx['Available Supply']
#    eligibility['Price'] = onchainfx['Price']
#    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
#    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
#    for t in eligibility.index:
#        if t in coinmarketcap.index:
#            eligibility.loc[t,('Volume')] = coinmarketcap.loc[t,('Volume (30d)')]
#        else:
#            print (t,' no volume data!')
#            eligibility.loc[t,('Volume')] = 1
#    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
#    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
#    for t in eligibility.index:
#        if t in assets.index:
#            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
#            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
#            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
#        else:
#            print (t,' no other data!')
#            eligibility.loc[t,('Other Pass')] = 0
#    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
#    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
#    check if duplicates
#    count = len(eligibility.index)
#    count1 = len(eligibility.index.unique())
#    if count != count1:
#        print ('duplicate tickers')
#    return eligibility

In [23]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [24]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [25]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

duplicate tickers


In [77]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check
ineligible_db

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
ANT,3.335032e+07,2.542650e+07,0.762407,3.019856e+07,0.841977,3.960952e+07,2.279824e+05,1,0.008966,0,1.0,3.0,1.0,2.0,0
BCN,1.064671e+08,1.062359e+08,0.997828,1.840668e+11,0.000577,1.844675e+11,2.859959e+05,1,0.002692,0,0.0,1.0,0.0,1.0,0
BLOCK,3.968202e+07,1.087846e+07,0.274141,5.974559e+06,1.820798,2.179375e+07,0.000000e+00,1,0.000000,0,1.0,2.0,0.0,2.0,0
BLZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,2.0,0.0,1.0,0
BTCP,2.562703e+07,5.758171e+06,0.224691,4.721780e+06,1.219492,2.101452e+07,7.721025e+05,1,0.134088,1,0.0,1.0,0.0,2.0,0
BTM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,2.0,0.0,0.0,0
CENNZ,3.164738e+07,2.132052e+07,0.673690,8.084281e+08,0.026373,1.200000e+09,1.205789e+05,1,0.005656,0,0.0,3.0,0.0,1.0,0
CND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1.0,3.0,0.0,1.0,0
CNX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,0.0,3.0,0.0,0.0,0
CTXC,3.615004e+07,2.131992e+07,0.589762,1.497925e+08,0.142330,2.539879e+08,1.073482e+08,1,5.035113,1,0.0,2.0,0.0,2.0,0


In [27]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [78]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,2.032016e+11,1.728115e+11,0.850444,1.784770e+07,9682.565153,2.098634e+07,1.399323e+11,1,0.809739,1,1.0,1.0,0.0,3.0,1,1
XRP,3.199818e+10,1.371847e+10,0.428726,4.287265e+10,0.319982,1.000000e+11,6.123244e+09,1,0.446350,1,1.0,2.0,0.0,3.0,1,2
ETH,2.866174e+10,2.271936e+10,0.792672,1.071182e+08,212.096092,1.351356e+08,4.815743e+10,1,2.119665,1,1.0,2.0,0.0,3.0,1,3
XLM,1.198269e+10,1.652145e+09,0.137878,1.961769e+10,0.084217,1.422834e+11,8.341657e+08,1,0.504899,1,1.0,1.0,0.0,3.0,1,4
LTC,7.646114e+09,5.733137e+09,0.749811,6.289452e+07,91.154797,8.388054e+07,1.587331e+10,1,2.768695,1,1.0,1.0,0.0,3.0,1,5
BCH,6.927756e+09,5.916803e+09,0.854072,1.792051e+07,330.169310,2.098243e+07,8.514442e+09,1,1.439027,1,1.0,1.0,0.0,3.0,1,6
EOS,5.804712e+09,3.931303e+09,0.677261,9.254170e+08,4.248143,1.366412e+09,1.323322e+10,1,3.366114,1,1.0,2.0,0.0,3.0,1,7
BSV,3.149642e+09,2.680222e+09,0.850961,1.785499e+07,150.110586,2.098214e+07,3.866938e+09,1,1.442768,1,1.0,1.0,0.0,3.0,1,8
BNB,2.699834e+09,4.199233e+09,1.555367,1.555367e+08,26.998340,1.000000e+08,2.093409e+09,1,0.498522,1,1.0,3.0,1.0,3.0,1,9
ADA,2.539518e+09,1.567149e+09,0.617105,2.592707e+10,0.060445,4.201402e+10,6.677349e+08,1,0.426082,1,1.0,2.0,0.0,3.0,1,10


In [66]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained ADA
Retained TRX
10 full! Do not add  BNB
10 full! Do not add  XTZ


In [67]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained BTG
Retained BAT
Retained MANA
Retained LSK
20 full! Do not add  QTUM
20 full! Do not add  SC
20 full! Do not add  ZIL
20 full! Do not add  ZRX


In [68]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained NAS
Retained PIVX
Retained POWR
Retained STORJ
Retained GNT
Retained STRAT
New add AION
New add ENG
New add ARDR
New add LRC
40 full! Do not add  IOTX
40 full! Do not add  WAN
40 full! Do not add  EDO
40 full! Do not add  POLY
40 full! Do not add  CVC
40 full! Do not add  PPT
40 full! Do not add  LOOM
40 full! Do not add  TNT
40 full! Do not add  BNT
40 full! Do not add  PAY


In [69]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [70]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [71]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

40 adds ['AION' 'ARDR' 'ENG' 'LRC']
40 deletes ['DGB' 'HOLO' 'RDD' 'ZEN']
40 count members: 40

ETH adds ['AION' 'ENG' 'LRC']
ETH deletes []
ETH count members: 23

Platform adds ['AION' 'ARDR']
Platform deletes ['DCR' 'HOLO']
Platform count members: 28

10 adds []
10 deletes []
10 count members: 10

Application adds ['ENG' 'ICX' 'LRC']
Application deletes ['RDD']
Application count members: 25

Total adds ['AION' 'ARDR' 'ENG' 'ICX' 'LRC']
Total deletes ['DCR' 'DGB' 'HOLO' 'RDD' 'ZEN']
Total count members: 70

20 adds ['ICX']
20 deletes ['DCR']
20 count members: 20

Currency adds []
Currency deletes ['DGB' 'ZEN']
Currency count members: 17

Sector count members: 70


In [72]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])